**For this part of the exam, you will create a Streamlit app that will allow users to predict the price of a home by inputting certain information about it.  It will include inputs for features of the home and produce a predicted price.**

Load in the provided ML model (part2-model-pipeline.joblib) and training data (part2-training-data.joblib) to determine which features were used and the range of values included in each feature. If you receive an error about no module named dill, run “!pip install dill” in your notebook before using joblib.load

In [2]:
pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import joblib
training_data = joblib.load('Models/part2-training-data.joblib')



In [13]:
training_data

[            bedrooms  bathrooms  sqft_living
 id                                          
 2473372170         4       3.25         2820
 597000195          3       1.75         1460
 4346300010         3       2.50         1560
 629650030          4       2.50         2233
 713500030          5       3.50         4800
 ...              ...        ...          ...
 2524049250         5       2.25         3270
 124069032          3       1.75         1670
 3825500020         4       2.50         3350
 7504110030         4       2.50         3300
 7806450190         3       2.50         2760
 
 [15905 rows x 3 columns],
 id
 2473372170     432000.0
 597000195      527200.0
 4346300010     545500.0
 629650030      317500.0
 713500030     1350000.0
                 ...    
 2524049250    1180000.0
 124069032      600000.0
 3825500020     550000.0
 7504110030     785000.0
 7806450190     500000.0
 Name: price, Length: 15905, dtype: float64]

In [14]:
model_pipeline = joblib.load('Models/part2-model-pipeline.joblib')

In [16]:
model_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x284b8b580>),
                                                 ('num', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x284b8b2b0>)],
                                   verbose_feature_names_out=False)),
                ('reg', LinearRegression())])